# Deep Dive into LangChain
## LLMs, Prompt Templates, Caching, Streaming, Chains

This notebook uses the latest versions of the OpenAI and LangChain libraries.

### Python-dotenv

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the API Keys from .env
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

## Chat Models: GPT-3.5 Turbo and GPT-4

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()  

# invoking the llm (running the prompt)
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo', temperature=0.1)
print(output.content)

Quantum mechanics is the branch of physics that describes the behavior of particles at the smallest scales, where traditional laws of physics no longer apply and phenomena such as superposition and entanglement occur.


In [3]:
# help(ChatOpenAI)  # see the llm constructor arguments with its defaults

In [4]:
# using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(
    SystemMessage, 
    AIMessage,
    HumanMessage
)
messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene durch Wellenfunktionen und Wahrscheinlichkeitsverteilungen.


## Caching LLM Responses

### 1. In-Memory Cache

In [5]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [6]:
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())

In [7]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

CPU times: user 17.5 ms, sys: 2.18 ms, total: 19.7 ms
Wall time: 934 ms


'\n\nWhy was the math book sad? Because it had too many problems!'

In [8]:
%%time
llm.invoke(prompt)

CPU times: user 373 µs, sys: 1e+03 ns, total: 374 µs
Wall time: 379 µs


'\n\nWhy was the math book sad? Because it had too many problems!'

### 2. SQLite Caching

In [9]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [10]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

CPU times: user 20.9 ms, sys: 3.56 ms, total: 24.5 ms
Wall time: 778 ms


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired!"

In [11]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

CPU times: user 60.6 ms, sys: 28.1 ms, total: 88.7 ms
Wall time: 88 ms


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired!"

## LLM Streaming

In [12]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

Verse 1:
Beneath the silver light of the moon
A raven flies high, singing a haunting tune
Its wings black as night, against the pale sky
A messenger of darkness, soaring up so high

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, forever in flight
Whispers of mystery, in the cold night air
Moon and raven, a timeless pair

Verse 2:
The moon watches over, with a steady gaze
As the raven calls out, in a mysterious haze
Their connection unbreakable, their bond so strong
In the darkness of the night, they both belong

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, forever in flight
Whispers of mystery, in the cold night air
Moon and raven, a timeless pair

Bridge:
The raven croaks out a solemn cry
Echoing through the night sky
The moon shines down, a guiding light
Together they soar, in the darkness of night

Chorus:
Moon and raven, dance in the night
Their spirits intertwined, forever in flight
Whispers of mystery, in the cold night air
Moon and

In [13]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

Verse 1:
Beneath the silver moon's bright glow
A raven flies, it's time to show
The secrets of the night unfold
As the raven's wings take hold

Chorus:
Moonlight shining through the night
Raven's wings take flight
In the darkness, they unite
A mystical sight, the moon and the raven

Verse 2:
The raven calls with a haunting cry
As the moon watches from the sky
Their bond is strong, their spirits free
Together they roam, wild and carefree

Chorus:
Moonlight shining through the night
Raven's wings take flight
In the darkness, they unite
A mystical sight, the moon and the raven

Bridge:
In the stillness of the night
They dance in the pale moonlight
A mesmerizing sight to see
The moon and the raven, wild and free

Chorus:
Moonlight shining through the night
Raven's wings take flight
In the darkness, they unite
A mystical sight, the moon and the raven

Outro:
As the night fades into day
The moon and the raven fly away
But their bond will never break
For in the night, they will awake.

## PromptTemplates

In [14]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: virus and language
prompt = prompt_template.format(virus='hiv', language='german')
prompt  # Returns the generated prompt


'You are an experienced virologist.\nWrite a few sentences about the following virus "hiv" in german.'

In [15]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

HIV, das humane Immundefizienzvirus, ist ein Retrovirus, das das menschliche Immunsystem schwächt und zu AIDS führen kann. Es wird hauptsächlich durch den Kontakt mit infiziertem Blut, sexuellen Kontakt oder von der Mutter auf das Kind übertragen. Die Behandlung von HIV hat sich in den letzten Jahren erheblich verbessert, aber es gibt immer noch keine Heilung für die Krankheit. Es ist wichtig, sich über die Übertragungswege und Präventionsmaßnahmen von HIV bewusst zu sein, um die Verbreitung der Krankheit einzudämmen.


## ChatPromptTemplates

In [16]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')
print(messages)  # Outputs the formatted chat messages


[SystemMessage(content='You respond only in the JSON format.'), HumanMessage(content='Top 5 countries in World by population.')]


In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()
output = llm.invoke(messages)
print(output.content)

{
    "countries": [
        {
            "name": "China",
            "population": "1,439,323,776"
        },
        {
            "name": "India",
            "population": "1,380,004,385"
        },
        {
            "name": "United States",
            "population": "331,002,651"
        },
        {
            "name": "Indonesia",
            "population": "273,523,615"
        },
        {
            "name": "Pakistan",
            "population": "220,892,340"
        }
    ]
}


## Simple Chains

In [18]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})




> Entering new LLMChain chain...
Prompt after formatting:
You are an experience virologist.
Write a few sentences about the following virus "HSV" in Spanish.


/opt/anaconda3/envs/langchain-pinecone/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(



> Finished chain.


In [19]:
print(output)

{'virus': 'HSV', 'language': 'Spanish', 'text': 'El virus del herpes simple (HSV) es una infección viral común que afecta a millones de personas en todo el mundo. Hay dos tipos de HSV: HSV-1, que suele causar herpes labial, y HSV-2, que suele causar herpes genital. Ambos tipos de virus pueden ser transmitidos a través del contacto directo con una persona infectada, y pueden causar síntomas como ampollas, úlceras y dolor en la piel. Aunque no existe cura para el HSV, existen tratamientos disponibles para controlar los síntomas y prevenir brotes recurrentes.'}


In [20]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke(country)
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:
What is the capital of ?. List the top 3 places to visit in that city. Use bullet points

> Finished chain.
I'm sorry, but you have not provided the name of the city for which you want to know the capital and top 3 places to visit. Please specify the city so I can provide you with the information you need.


## Sequential Chains

In [21]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)
# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain with the concept "linear regression"
output = overall_chain.invoke('linear regression')




> Entering new SimpleSequentialChain chain...
Sure! Here is a simple implementation of linear regression in Python:

```python
import numpy as np

def linear_regression(X, y):
    # Calculate the number of data points
    n = len(X)
    
    # Calculate the mean of X and y
    mean_X = np.mean(X)
    mean_y = np.mean(y)
    
    # Calculate the slope (m) and y-intercept (b) of the regression line
    numerator = np.sum((X - mean_X) * (y - mean_y))
    denominator = np.sum((X - mean_X) ** 2)
    m = numerator / denominator
    b = mean_y - m * mean_X
    
    return m, b

# Example usage
X = np.array([1, 2, 3, 4, 5])
y = np.array([2, 4, 5, 4, 5])

m, b = linear_regression(X, y)
print("Slope:", m)
print("Y-intercept:", b)
```

This function takes in two arrays X and y, where X represents the independent variable and y represents the dependent variable. It calculates the slope and y-intercept of the regression line that best fits the data points using the least squares method.
The provi

In [22]:
print(output['output'])

The provided Python function, `linear_regression(X, y)`, is an implementation of simple linear regression using the method of least squares. Linear regression is a fundamental statistical approach for modeling the relationship between an independent variable (or variables, in the case of multiple regression) and a dependent variable. This particular function assumes a single independent variable, making it a model for simple linear regression. The relationship is modeled through a straight line with the equation \(y = mx + b\), where:

- \(y\) is the estimated dependent variable (target),
- \(x\) is the independent variable (predictor),
- \(m\) is the slope of the regression line (representing the change in \(y\) for a one-unit change in \(x\)),
- \(b\) is the y-intercept (the value of \(y\) when \(x = 0\)).

Here's a detailed explanation of each step in the function:

### Import Necessary Library
- The function begins by importing NumPy, which is a popular Python library for numerical

In [2]:
from langchain_experimental.utilities import PythonREPL
python_repl = PythonREPL()
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

'[13, 26, 39, 52, 65, 78, 91]\n'

In [3]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
agent_executor = create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

# Invoke the agent
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'
agent_executor.invoke(prompt)



> Entering new AgentExecutor chain...
To solve this, I need to calculate the factorial of 12 first, then find its square root, and finally format the result to display it with 4 decimal points. I can use the `math` module in Python for both the factorial and square root calculations, and then format the output as required.
Action: Python_REPL
Action Input: import math
Observation: 
Thought:Now that the math module is imported, I can proceed to calculate the factorial of 12.
Action: Python_REPL
Action Input: factorial_12 = math.factorial(12)
Observation: 
Thought:I have calculated the factorial of 12. Next, I need to calculate the square root of this value.
Action: Python_REPL
Action Input: sqrt_factorial_12 = math.sqrt(factorial_12)
Observation: 
Thought:I have calculated the square root of the factorial of 12. Now, I need to format this result to display it with 4 decimal points.
Action: Python_REPL
Action Input: print(f"{sqrt_factorial_12:.4f}")
Observation: 21886.1052

Thought:I n

{'input': 'Calculate the square root of the factorial of 12 and display it with 4 decimal points',
 'output': '21886.1052'}

In [4]:
response = agent_executor.invoke('What is the answer to 5.1 ** 7.3?')



> Entering new AgentExecutor chain...
I need to calculate 5.1 raised to the power of 7.3 to get the answer.
Action: Python_REPL
Action Input: print(5.1 ** 7.3)
Observation: 146306.05007233328

Thought:I now know the final answer
Final Answer: 146306.05007233328

> Finished chain.


In [5]:
response

{'input': 'What is the answer to 5.1 ** 7.3?', 'output': '146306.05007233328'}

In [6]:
print(response['input'])

What is the answer to 5.1 ** 7.3?


In [7]:
print(response['output'])


146306.05007233328
